In [46]:
#!/usr/bin/env python3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import collections

In [115]:
MRR = pd.read_csv('../data/clinical_analysis_data/manual_rereview.txt', sep='\t')

In [147]:
consensus_table = {'S':[['S','S','S','S','S','S','S'],
['S','S','S','S','S','S','A'],
['S','S','S','S','S','A','A',],
['S','S','S','S','S','S','F'],
['S','S','S','S','S','A','F'],
['S','S','S','S','S','F','F'],
['S','S','S','S','A','A','F'],
['S','S','S','S','A','F','F']],
'No Consensus':[['S','S','S','S','A','A','A'],
['S','S','S','S','F','F','F'],
['F','F','F','F','A','A','A'],
['F','F','F','F','S','S','S'],
['A','A','A','A','F','F','F'],
['A','A','A','A','S','S','S'],
['A','S','S','S','F','F','F'],
['A','A','F','F','F','S','S'],
['S','S','S','A','A','A','F'],
['S','S','F','F','A','A','A'],
['F','F','F','A','A','A','S'],
['S','S','S','F','F','A','A']],
'F':[['F','F','F','F','F','F','F'],
['F','F','F','F','F','F','A'],
['F','F','F','F','F','F','S'],
['F','F','F','F','F','A','S'],
['F','F','F','F','F','S','S'],
['F','F','F','F','A','A','S'],
['F','F','F','F','A','S','S'],
['F','F','F','F','F','A','A']],
'A':[['A','A','A','A','A','A','A'],
['A','A','A','A','A','A','F'],
['A','A','A','A','A','A','S'],
['A','A','A','A','A','F','S'],
['A','A','A','A','A','S','S'],
['A','A','A','A','F','F','S'],
['A','A','A','A','F','S','S']]}

In [148]:
for i,row in MRR.iterrows():
    variant_calls = []
    for item in row[8:15]:
        variant_calls.append(item)
    for k,thing in consensus_table.items():
        for item in thing:
            if sorted(variant_calls) == sorted(item):
                MRR.loc[i, 'Consensus'] = k

In [154]:
MR_count = 0
classifier_count = 0
new_consensus = 0
no_consensus = 0

for i,row in MRR.iterrows():
    if row['Consensus'] == 'No Consensus':
        no_consensus += 1
    elif row['Consensus'] == row['Classifier_Call']:
        classifier_count+=1
    elif row['Consensus'] == row['MR_Call']:
        MR_count+=1
    else:
        new_consensus += 1


In [155]:
# Print statements
total = len(MRR_new)
print('Total Variants: ',total)
print('MR-Variants: ',MR_count)
print('Classifier-Variants: ',classifier_count)
print('Change-Variants: ',new_consensus)
print('No Consensus: ',no_consensus)

Total Variants:  179
MR-Variants:  53
Classifier-Variants:  51
Change-Variants:  34
No Consensus:  41
